In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import timedelta 

In [2]:
fecha = pd.to_datetime('30/06/2020')

In [3]:
path_1 = 'df_sin_fallecidos_al2020.csv'

In [4]:
df = pd.read_csv(path_1, encoding = 'latin', index_col = 0)

In [5]:
df['FECHA_NACIMIENTO'] = pd.to_datetime(df['FECHA_NACIMIENTO'])
df['FECHA_TITULO'] = pd.to_datetime(df['FECHA_TITULO'])
df['FECHA_MATRICULA'] = pd.to_datetime(df['FECHA_MATRICULA'])
df['FECHA_FALLECIDO'] = pd.to_datetime(df['FECHA_FALLECIDO'])

# Ingreso de nuevos profesionales

In [6]:
df_mat = df.loc[(df['FECHA_MATRICULA']> fecha - pd.offsets.DateOffset(years=9))][['ID_PROFESIONAL_MATRICULA', 'FECHA_MATRICULA']]

In [7]:
df_mat

,ID_PROFESIONAL_MATRICULA,FECHA_MATRICULA
12,21462737,2014-10-20
16,21084021,2011-09-26
21,21674021,2017-02-01
72,21735263,2018-03-23
88,21735313,2018-04-06
...,...,...
1516543,21971235,2020-01-30
1516545,21987953,2019-10-03
1516546,21992790,2019-11-29
1516547,21994573,2019-10-07


In [8]:
#Obtenemos los datos y generamos el dataframe
#Fecha base y periodo
fecha = pd.to_datetime('2020-06-30')
anios = 10

#Generamos los periodos deseados
periodos = pd.date_range(end=fecha, periods=anios, freq=pd.DateOffset(years=1))

#Generamos las columnas deseadas
for periodo in periodos:
    #Creamos el título de la columna y la creamos con el método assign()
    col = 'matriculados_' + str(periodo.year)
    #Copiamos la fecha de fallecido en las columnas
    kwargs = {col : lambda x: x['FECHA_MATRICULA']}
    df_mat = df_mat.assign(**kwargs)
    
    #Sustituímos la fecha de fallecido con True o False dependiendo del periodo(Fecha de corte de la columna)
    df_mat[col] = df_mat[col].apply(lambda x: True if (x <= periodo) else False)

In [9]:
df_mat= pd.merge(df, df_mat, on = 'ID_PROFESIONAL_MATRICULA', how = 'left')

In [10]:
df_mat.iloc[:,-10:] = df_mat.iloc[:,-10:].fillna(True)

In [11]:
df_2 = df_mat.iloc[:,:-13]

In [12]:
df_mat = df_mat.groupby((['ID_PROFESIONAL_MATRICULA']))\
['matriculados_2011',
 'matriculados_2012',
 'matriculados_2013',
 'matriculados_2014', 
 'matriculados_2015', 
 'matriculados_2016', 
 'matriculados_2017', 
 'matriculados_2018',  
 'matriculados_2019', 
 'matriculados_2020'].sum().reset_index()



/tmp/ipykernel_15046/3022779030.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mat = df_mat.groupby((['ID_PROFESIONAL_MATRICULA']))\


In [13]:
df_mat.iloc[:,-10:] = df_mat.iloc[:,-10:].astype(bool)

In [14]:
df_mat = pd.merge(df_2, df_mat)

# Sanciones

In [15]:
path_2 = 'Bajada Sancion 29-04-2022.txt'

In [16]:
df_sanciones = pd.read_csv(path_2, sep = '|')

In [17]:
#Saco las sanciones no inhabilitantes (codigo 14 y 15) y las bajas por fallecimiento y jubilacion (codigo 6 y )

df_sanciones = df_sanciones.loc[(df_sanciones['ID_TIPO_SANCION']!=14) &
                                (df_sanciones['ID_TIPO_SANCION']!=15) &
                               (df_sanciones['ID_TIPO_SANCION']!=6) &
                               (df_sanciones['ID_TIPO_SANCION']!=7)]

In [18]:
df_sanciones

,ID_PROFESIONAL,ID_PROFESIONAL_PROFESION,ID_PROFESIONAL_MATRICULA,ID_PROFESIONAL_SANCION,ID_SITUACION_MATRICULA,FECHA_SANCION,FECHA_VIGENCIA,ID_TIPO_SANCION,ID_USUARIO_REGISTRO,FECHA_REGISTRO
10,36005456,20458264,20636581,659083,3.0,04/05/1987,NaN,8,582,01/08/2016
11,40004088,20631883,20636585,659084,3.0,01/12/2000,NaN,11,582,01/08/2016
12,32001657,410356,20636588,659082,3.0,01/12/2003,NaN,8,582,01/08/2016
13,20332873,20400275,20636591,659085,3.0,03/08/2004,NaN,11,582,01/08/2016
14,44008531,21090720,21101307,723623,1.0,29/10/2015,10/05/2018,8,3061,03/03/2017
...,...,...,...,...,...,...,...,...,...,...
117236,36052345,21679634,22037372,1134748,3.0,29/01/2021,NaN,9,9958,21/01/2022
117271,34008815,21276855,22086650,1184564,3.0,31/07/2014,NaN,8,53771,01/04/2022
117272,20315807,20979310,22086678,1184582,3.0,09/06/2014,NaN,8,53771,01/04/2022
117273,50026149,21302217,22086001,1187137,3.0,22/04/2022,NaN,8,23609,26/04/2022


In [19]:
df_sanciones['FECHA_VIGENCIA'] = pd.to_datetime(df_sanciones['FECHA_VIGENCIA'])
df_sanciones['FECHA_SANCION'] = pd.to_datetime(df_sanciones['FECHA_SANCION'])

In [20]:
df_sanciones[df_sanciones['FECHA_VIGENCIA'].notnull()]

,ID_PROFESIONAL,ID_PROFESIONAL_PROFESION,ID_PROFESIONAL_MATRICULA,ID_PROFESIONAL_SANCION,ID_SITUACION_MATRICULA,FECHA_SANCION,FECHA_VIGENCIA,ID_TIPO_SANCION,ID_USUARIO_REGISTRO,FECHA_REGISTRO
14,44008531,21090720,21101307,723623,1.0,2015-10-29,2018-10-05,8,3061,03/03/2017
102,44008528,21090717,21101304,392516,1.0,2014-04-25,2018-02-01,5,3061,26/08/2014
234,72009266,21112645,21134272,605916,1.0,2014-06-23,2017-06-14,5,2143,06/10/2015
241,72001966,20503875,21134345,256634,1.0,2010-07-29,2010-07-29,5,2143,10/10/2013
242,72009326,21112705,21134355,605917,1.0,2014-06-18,2018-09-08,5,2143,06/10/2015
...,...,...,...,...,...,...,...,...,...,...
115917,58015407,21568627,21930672,993358,1.0,2020-08-27,2020-08-27,8,2143,13/11/2020
115930,20915225,21624330,21914470,977037,1.0,2019-05-06,2019-05-06,8,2143,12/08/2020
115933,44017240,21663088,21970925,1033848,2.0,2021-01-28,2021-02-28,11,20567,23/02/2021
115934,44017228,21663091,21970927,1033851,1.0,2021-01-26,2021-01-26,11,20567,23/02/2021


In [21]:
tipos_sanciones = pd.read_csv('tipos_sanciones280422.csv', encoding ='latin')

In [22]:
tipos_sanciones

,Código,Nombre,ID_SITUACION_MATRICULA
0,1,Sanción sanitaria por incumplimiento administr...,3
1,2,Sanción sanitaria por mala praxis en el ejecrc...,3
2,3,Sanción administrativa por falta de cumplimien...,3
3,4,Sanción penal ajena al ejercicio de la profesión,3
4,5,Inhabilitación por motivos personales del prof...,3
5,6,Baja por fallecimiento,3
6,7,Baja por jubilación,3
7,8,Baja voluntaria,3
8,9,Baja administrativa,3
9,10,Baja por rematriculación,3


In [23]:
#convertir el DF a Diccionario utilizando zip

dict_sanciones = dict(zip(tipos_sanciones['Código'], tipos_sanciones['Nombre']))

In [24]:
df_sanciones['desc_sancion'] = df_sanciones['ID_TIPO_SANCION'].map(dict_sanciones)

In [25]:
df_sanciones = pd.merge(df_sanciones,df[['ID_PROFESIONAL','ID_PROFESIONAL_PROFESION','ID_PROFESIONAL_MATRICULA', 'ID_PROVINCIA_MATRICULA', 'ID_ORIGEN_EMITE','PROFESION_REFERENCIA']], on =['ID_PROFESIONAL','ID_PROFESIONAL_PROFESION','ID_PROFESIONAL_MATRICULA'], how = 'left')

In [26]:
df_sanciones

,ID_PROFESIONAL,ID_PROFESIONAL_PROFESION,ID_PROFESIONAL_MATRICULA,ID_PROFESIONAL_SANCION,ID_SITUACION_MATRICULA,FECHA_SANCION,FECHA_VIGENCIA,ID_TIPO_SANCION,ID_USUARIO_REGISTRO,FECHA_REGISTRO,desc_sancion,ID_PROVINCIA_MATRICULA,ID_ORIGEN_EMITE,PROFESION_REFERENCIA
0,36005456,20458264,20636581,659083,3.0,1987-04-05,NaT,8,582,01/08/2016,Baja voluntaria,8.0,0.0,Médico
1,40004088,20631883,20636585,659084,3.0,2000-01-12,NaT,11,582,01/08/2016,Baja Otros,8.0,0.0,Médico
2,32001657,410356,20636588,659082,3.0,2003-01-12,NaT,8,582,01/08/2016,Baja voluntaria,8.0,0.0,Médico
3,20332873,20400275,20636591,659085,3.0,2004-03-08,NaT,11,582,01/08/2016,Baja Otros,8.0,0.0,Médico
4,44008531,21090720,21101307,723623,1.0,2015-10-29,2018-10-05,8,3061,03/03/2017,Baja voluntaria,10.0,108334.0,Médico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71920,36052345,21679634,22037372,1134748,3.0,2021-01-29,NaT,9,9958,21/01/2022,Baja administrativa,3.0,108955.0,Médico
71921,34008815,21276855,22086650,1184564,3.0,2014-07-31,NaT,8,53771,01/04/2022,Baja voluntaria,23.0,0.0,Enfermero
71922,20315807,20979310,22086678,1184582,3.0,2014-09-06,NaT,8,53771,01/04/2022,Baja voluntaria,23.0,0.0,Médico
71923,50026149,21302217,22086001,1187137,3.0,2022-04-22,NaT,8,23609,26/04/2022,Baja voluntaria,NaN,NaN,NaN


# Eliminación de sanciones duplicadas para misma matrícula en mismo tiempo

In [27]:
df_sanciones['FECHA_VIGENCIA'].loc[(df_sanciones['FECHA_VIGENCIA'].isnull())] = df_sanciones['FECHA_VIGENCIA'].loc[(df_sanciones['FECHA_VIGENCIA'].isnull())].fillna(fecha+ timedelta(days=1))

/home/oblitterator/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [28]:
df_sanciones

,ID_PROFESIONAL,ID_PROFESIONAL_PROFESION,ID_PROFESIONAL_MATRICULA,ID_PROFESIONAL_SANCION,ID_SITUACION_MATRICULA,FECHA_SANCION,FECHA_VIGENCIA,ID_TIPO_SANCION,ID_USUARIO_REGISTRO,FECHA_REGISTRO,desc_sancion,ID_PROVINCIA_MATRICULA,ID_ORIGEN_EMITE,PROFESION_REFERENCIA
0,36005456,20458264,20636581,659083,3.0,1987-04-05,2020-07-01,8,582,01/08/2016,Baja voluntaria,8.0,0.0,Médico
1,40004088,20631883,20636585,659084,3.0,2000-01-12,2020-07-01,11,582,01/08/2016,Baja Otros,8.0,0.0,Médico
2,32001657,410356,20636588,659082,3.0,2003-01-12,2020-07-01,8,582,01/08/2016,Baja voluntaria,8.0,0.0,Médico
3,20332873,20400275,20636591,659085,3.0,2004-03-08,2020-07-01,11,582,01/08/2016,Baja Otros,8.0,0.0,Médico
4,44008531,21090720,21101307,723623,1.0,2015-10-29,2018-10-05,8,3061,03/03/2017,Baja voluntaria,10.0,108334.0,Médico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71920,36052345,21679634,22037372,1134748,3.0,2021-01-29,2020-07-01,9,9958,21/01/2022,Baja administrativa,3.0,108955.0,Médico
71921,34008815,21276855,22086650,1184564,3.0,2014-07-31,2020-07-01,8,53771,01/04/2022,Baja voluntaria,23.0,0.0,Enfermero
71922,20315807,20979310,22086678,1184582,3.0,2014-09-06,2020-07-01,8,53771,01/04/2022,Baja voluntaria,23.0,0.0,Médico
71923,50026149,21302217,22086001,1187137,3.0,2022-04-22,2020-07-01,8,23609,26/04/2022,Baja voluntaria,NaN,NaN,NaN


In [29]:
fecha = pd.to_datetime('2020-06-30')
anios = 10

#Generamos los periodos deseados
periodos = pd.date_range(end=fecha, periods=anios, freq=pd.DateOffset(years=1))

#Generamos las columnas deseadas
for periodo in periodos:
    #Creamos el título de la columna y la creamos con el método assign()
    col = 'mat_activa_' + str(periodo.year)
    
    #Copiamos la fecha de fallecido en las columnas
    kwargs = {col : lambda x: ( x['FECHA_SANCION'] > periodo) | (x['FECHA_VIGENCIA'] < periodo)} #calculo el intervalo de inactividad segun fecha de vigencia
    df_sanciones = df_sanciones.assign(**kwargs)
    df_sanciones[col] = df_sanciones[col] == True

In [30]:

df_sanciones = df_sanciones.groupby((['ID_PROFESIONAL_MATRICULA','FECHA_SANCION','FECHA_VIGENCIA']))\
['mat_activa_2011',
 'mat_activa_2012',
 'mat_activa_2013',
 'mat_activa_2014', 
 'mat_activa_2015', 
 'mat_activa_2016', 
 'mat_activa_2017', 
 'mat_activa_2018',  
 'mat_activa_2019', 
 'mat_activa_2020'].sum().reset_index()

/tmp/ipykernel_15046/1499473820.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_sanciones = df_sanciones.groupby((['ID_PROFESIONAL_MATRICULA','FECHA_SANCION','FECHA_VIGENCIA']))\


In [31]:
df_sanciones

,ID_PROFESIONAL_MATRICULA,FECHA_SANCION,FECHA_VIGENCIA,mat_activa_2011,mat_activa_2012,mat_activa_2013,mat_activa_2014,mat_activa_2015,mat_activa_2016,mat_activa_2017,mat_activa_2018,mat_activa_2019,mat_activa_2020
0,100252,2014-12-16,2020-07-01,1,1,1,1,0,0,0,0,0,0
1,100679,2015-07-28,2020-07-01,1,1,1,1,1,0,0,0,0,0
2,101017,2001-06-02,2020-07-01,0,0,0,0,0,0,0,0,0,0
3,309735,2019-08-10,2020-08-10,1,1,1,1,1,1,1,1,1,0
4,310357,2014-11-04,2020-07-01,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67312,22086682,2014-09-06,2020-07-01,1,1,1,1,0,0,0,0,0,0
67313,22086755,2015-08-24,2020-07-01,1,1,1,1,1,0,0,0,0,0
67314,22086819,2021-03-11,2020-07-01,1,1,1,1,1,1,1,1,1,1
67315,22088458,2022-02-02,2020-07-01,1,1,1,1,1,1,1,1,1,1


In [32]:
df_sanciones.iloc[:,-10:] = df_sanciones.iloc[:,-10:].astype(bool)

In [33]:
df_sanciones = df_sanciones.drop_duplicates('ID_PROFESIONAL_MATRICULA')

In [34]:
df_sanciones[['mat_activa_2011',
 'mat_activa_2012',
 'mat_activa_2013',
 'mat_activa_2014', 
 'mat_activa_2015', 
 'mat_activa_2016', 
 'mat_activa_2017', 
 'mat_activa_2018',  
 'mat_activa_2019', 
 'mat_activa_2020']] = df_sanciones[['mat_activa_2011',
 'mat_activa_2012',
 'mat_activa_2013',
 'mat_activa_2014', 
 'mat_activa_2015', 
 'mat_activa_2016', 
 'mat_activa_2017', 
 'mat_activa_2018',  
 'mat_activa_2019', 
 'mat_activa_2020']]

/home/oblitterator/.local/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [35]:
df_sanciones = df_sanciones[['ID_PROFESIONAL_MATRICULA','mat_activa_2011',
 'mat_activa_2012',
 'mat_activa_2013',
 'mat_activa_2014', 
 'mat_activa_2015', 
 'mat_activa_2016', 
 'mat_activa_2017', 
 'mat_activa_2018',  
 'mat_activa_2019', 
 'mat_activa_2020']]

In [36]:
df_mat

,ID_PROFESIONAL,CODIGO,NOMBRE,APELLIDO,ID_TIPODOC,NRODOC,FECHA_NACIMIENTO,ID_PAIS_NACIMIENTO,SEXO,CUIL,...,matriculados_2011,matriculados_2012,matriculados_2013,matriculados_2014,matriculados_2015,matriculados_2016,matriculados_2017,matriculados_2018,matriculados_2019,matriculados_2020
0,10000000,541010896218,MARIA JULIA,VELEZ,1,10896218,1953-07-14,0,F,27108962181,...,True,True,True,True,True,True,True,True,True,True
1,10000001,541013388083,MARIA CRISTINA,ANFOLISI,1,13388083,1959-05-05,0,F,27133880831,...,True,True,True,True,True,True,True,True,True,True
2,10000002,541005910020,ELSA SUSANA,ORELLANA,1,5910020,1948-05-12,0,F,27059100209,...,True,True,True,True,True,True,True,True,True,True
3,10000003,541012071879,MONICA FLORENCIA,ROBERT,1,12071879,1958-06-04,0,F,27120718792,...,True,True,True,True,True,True,True,True,True,True
4,10000003,541012071879,MONICA FLORENCIA,ROBERT,1,12071879,1958-06-04,0,F,27120718792,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444773,82000654,541037969493,CAROLINA,SUAU,1,37969493,1993-12-13,200,F,23379694934,...,False,False,False,False,False,False,False,False,False,True
1444774,82000694,541033225210,DAVID RAMON,SAMUDIO,1,33225210,1987-09-28,200,M,20332252101,...,False,False,False,False,False,False,False,False,False,True
1444775,82000714,541039056466,SOL DANIELA,BUSCIO,1,39056466,1995-05-09,200,F,27390564665,...,False,False,False,False,False,False,False,False,False,True
1444776,82000734,541093441325,ROSA LUZ,FENCO SILVA,1,93441325,1965-07-12,222,F,27934413259,...,False,False,False,False,False,False,False,False,False,True


In [37]:
df_sanciones = pd.merge(df_sanciones, df_mat[['ID_PROFESIONAL_MATRICULA','matriculados_2011',
 'matriculados_2012',
 'matriculados_2013',
 'matriculados_2014', 
 'matriculados_2015', 
 'matriculados_2016', 
 'matriculados_2017', 
 'matriculados_2018',  
 'matriculados_2019', 
 'matriculados_2020']], on = 'ID_PROFESIONAL_MATRICULA', how = 'right' )

In [38]:
#df_sanciones.iloc[:,-10:] = df_sanciones.iloc[:,-10:].fillna(True)

In [39]:
len(df_sanciones)

1444778

In [40]:
True == False


False

In [41]:
df_sanciones['mat_activa_2011'] = df_sanciones['mat_activa_2011'].fillna(df_sanciones['matriculados_2011'])
df_sanciones['mat_activa_2012'] = df_sanciones['mat_activa_2012'].fillna(df_sanciones['matriculados_2012'])
df_sanciones['mat_activa_2013'] = df_sanciones['mat_activa_2013'].fillna(df_sanciones['matriculados_2013'])
df_sanciones['mat_activa_2014'] = df_sanciones['mat_activa_2014'].fillna(df_sanciones['matriculados_2014'])
df_sanciones['mat_activa_2015'] = df_sanciones['mat_activa_2015'].fillna(df_sanciones['matriculados_2015'])
df_sanciones['mat_activa_2016'] = df_sanciones['mat_activa_2016'].fillna(df_sanciones['matriculados_2016'])
df_sanciones['mat_activa_2017'] = df_sanciones['mat_activa_2017'].fillna(df_sanciones['matriculados_2017'])
df_sanciones['mat_activa_2018'] = df_sanciones['mat_activa_2018'].fillna(df_sanciones['matriculados_2018'])
df_sanciones['mat_activa_2019'] = df_sanciones['mat_activa_2019'].fillna(df_sanciones['matriculados_2019'])
df_sanciones['mat_activa_2020'] = df_sanciones['mat_activa_2020'].fillna(df_sanciones['matriculados_2020'])

In [42]:
df_sanciones = df_sanciones.iloc[:,:-10]

# Fallecidos por año

In [43]:
df_fall = df.sort_values('ID_PROFESIONAL_MATRICULA').loc[df['FECHA_FALLECIDO'].notnull()][['ID_PROFESIONAL', 'FECHA_FALLECIDO']]

In [44]:
df_fall

,ID_PROFESIONAL,FECHA_FALLECIDO
1215925,42000446,2000-11-01
236604,20011410,2017-06-29
1215928,42000448,2002-08-06
1215378,42000002,2016-05-03
1215923,42000444,2018-10-20
...,...,...
1217303,42001564,2017-09-14
1436941,66016877,2020-10-02
277456,20042976,2020-04-27
462711,20178218,2011-06-30


In [45]:
len(df_fall)

91958

In [46]:
#Obtenemos los datos y generamos el dataframe
#Fecha base y periodo
fecha = pd.to_datetime('2020-06-30')
anios = 10

#Generamos los periodos deseados
periodos = pd.date_range(end=fecha, periods=anios, freq=pd.DateOffset(years=1))

#Generamos las columnas deseadas
for periodo in periodos:
    #Creamos el título de la columna y la creamos con el método assign()
    col = 'no_fallecido_' + str(periodo.year)
    #Copiamos la fecha de fallecido en las columnas
    kwargs = {col : lambda x: x['FECHA_FALLECIDO']}
    df_fall = df_fall.assign(**kwargs)
    
    #Sustituímos la fecha de fallecido con True o False dependiendo del periodo(Fecha de corte de la columna)
    df_fall[col] = df_fall[col].apply(lambda x: False if (x <= periodo) else True)

In [47]:
df_fall.head()

,ID_PROFESIONAL,FECHA_FALLECIDO,no_fallecido_2011,no_fallecido_2012,no_fallecido_2013,no_fallecido_2014,no_fallecido_2015,no_fallecido_2016,no_fallecido_2017,no_fallecido_2018,no_fallecido_2019,no_fallecido_2020
1215925,42000446,2000-11-01,False,False,False,False,False,False,False,False,False,False
236604,20011410,2017-06-29,True,True,True,True,True,True,False,False,False,False
1215928,42000448,2002-08-06,False,False,False,False,False,False,False,False,False,False
1215378,42000002,2016-05-03,True,True,True,True,True,False,False,False,False,False
1215923,42000444,2018-10-20,True,True,True,True,True,True,True,True,False,False


In [48]:
df_fall = pd.merge(df, df_fall.drop_duplicates('ID_PROFESIONAL'), on = 'ID_PROFESIONAL', how = 'left')

In [49]:
df_fall.iloc[:,-10:] = df_fall.iloc[:,-10:].fillna(True)

In [50]:
len(df_fall)

1444778

In [51]:
df_fall['no_fallecido_2015'].value_counts()

True     1395628
False      49150
Name: no_fallecido_2015, dtype: int64

# Inactivos por anio (mayores de 64)

In [52]:
df_jubilacion = df[['ID_PROFESIONAL', 'FECHA_NACIMIENTO']]

In [53]:
df_jubilacion.head()

,ID_PROFESIONAL,FECHA_NACIMIENTO
0,10000000,1953-07-14
1,10000001,1959-05-05
2,10000002,1948-05-12
3,10000003,1958-06-04
4,10000003,1958-06-04


In [54]:
df_jubilacion['FECHA_INACTIVO'] = df_jubilacion['FECHA_NACIMIENTO'] + pd.offsets.DateOffset(years=65)

/tmp/ipykernel_15046/1611704576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jubilacion['FECHA_INACTIVO'] = df_jubilacion['FECHA_NACIMIENTO'] + pd.offsets.DateOffset(years=65)


In [55]:
df_jubilacion = df_jubilacion.loc[df_jubilacion['FECHA_INACTIVO']<fecha]

In [56]:
#Obtenemos los datos y generamos el dataframe
#Fecha base y periodo
fecha = pd.to_datetime('2020-06-30')
anios = 10

#Generamos los periodos deseados
periodos = pd.date_range(end=fecha, periods=anios, freq=pd.DateOffset(years=1))

#Generamos las columnas deseadas
for periodo in periodos:
    #Creamos el título de la columna y la creamos con el método assign()
    col = 'no_jubilado_' + str(periodo.year)
    #Copiamos la fecha de fallecido en las columnas
    kwargs = {col : lambda x: x['FECHA_INACTIVO']}
    df_jubilacion = df_jubilacion.assign(**kwargs)
    
    #Sustituímos la fecha de fallecido con True o False dependiendo del periodo(Fecha de corte de la columna)
    df_jubilacion[col] = df_jubilacion[col].apply(lambda x: False if (x <= periodo) else True)

In [57]:
df_jubilacion.head()

,ID_PROFESIONAL,FECHA_NACIMIENTO,FECHA_INACTIVO,no_jubilado_2011,no_jubilado_2012,no_jubilado_2013,no_jubilado_2014,no_jubilado_2015,no_jubilado_2016,no_jubilado_2017,no_jubilado_2018,no_jubilado_2019,no_jubilado_2020
0,10000000,1953-07-14,2018-07-14,True,True,True,True,True,True,True,True,False,False
2,10000002,1948-05-12,2013-05-12,True,True,False,False,False,False,False,False,False,False
12,10000014,1951-04-19,2016-04-19,True,True,True,True,True,False,False,False,False,False
14,10000016,1953-03-25,2018-03-25,True,True,True,True,True,True,True,False,False,False
15,10000017,1951-09-01,2016-09-01,True,True,True,True,True,True,False,False,False,False


In [58]:
df_jubilacion= pd.merge(df, df_jubilacion.drop_duplicates('ID_PROFESIONAL'), on = 'ID_PROFESIONAL', how = 'left')

In [59]:
len(df_jubilacion)

1444778

In [60]:
df_jubilacion.iloc[:,-10:] = df_jubilacion.iloc[:,-10:].fillna(True)

In [61]:
df_jubilacion

,ID_PROFESIONAL,CODIGO,NOMBRE,APELLIDO,ID_TIPODOC,NRODOC,FECHA_NACIMIENTO_x,ID_PAIS_NACIMIENTO,SEXO,CUIL,...,no_jubilado_2011,no_jubilado_2012,no_jubilado_2013,no_jubilado_2014,no_jubilado_2015,no_jubilado_2016,no_jubilado_2017,no_jubilado_2018,no_jubilado_2019,no_jubilado_2020
0,10000000,541010896218,MARIA JULIA,VELEZ,1,10896218,1953-07-14,0,F,27108962181,...,True,True,True,True,True,True,True,True,False,False
1,10000001,541013388083,MARIA CRISTINA,ANFOLISI,1,13388083,1959-05-05,0,F,27133880831,...,True,True,True,True,True,True,True,True,True,True
2,10000002,541005910020,ELSA SUSANA,ORELLANA,1,5910020,1948-05-12,0,F,27059100209,...,True,True,False,False,False,False,False,False,False,False
3,10000003,541012071879,MONICA FLORENCIA,ROBERT,1,12071879,1958-06-04,0,F,27120718792,...,True,True,True,True,True,True,True,True,True,True
4,10000003,541012071879,MONICA FLORENCIA,ROBERT,1,12071879,1958-06-04,0,F,27120718792,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444773,82000654,541037969493,CAROLINA,SUAU,1,37969493,1993-12-13,200,F,23379694934,...,True,True,True,True,True,True,True,True,True,True
1444774,82000694,541033225210,DAVID RAMON,SAMUDIO,1,33225210,1987-09-28,200,M,20332252101,...,True,True,True,True,True,True,True,True,True,True
1444775,82000714,541039056466,SOL DANIELA,BUSCIO,1,39056466,1995-05-09,200,F,27390564665,...,True,True,True,True,True,True,True,True,True,True
1444776,82000734,541093441325,ROSA LUZ,FENCO SILVA,1,93441325,1965-07-12,222,F,27934413259,...,True,True,True,True,True,True,True,True,True,True


In [62]:
df_jubilacion['no_jubilado_2015'].value_counts()

True     1247405
False     197373
Name: no_jubilado_2015, dtype: int64

In [63]:
df_jubilacion.columns

Index(['ID_PROFESIONAL', 'CODIGO', 'NOMBRE', 'APELLIDO', 'ID_TIPODOC',
       'NRODOC', 'FECHA_NACIMIENTO_x', 'ID_PAIS_NACIMIENTO', 'SEXO', 'CUIL',
       'ID_PROVINCIA_RENAPER', 'PROVINCIA_CMDB', 'ID_CMDB_PERSONA',
       'FECHA_FALLECIDO', 'ID_PROFESIONAL_PROFESION',
       'ID_PROFESION_REFERENCIA', 'PROFESION_REFERENCIA',
       'ID_INSTITUCION_FORMADORA', 'DENOMINACION', 'FECHA_TITULO',
       'ID_PROFESIONAL_MATRICULA', 'MATRICULA', 'FECHA_MATRICULA',
       'REMATRICULACION', 'ID_PROVINCIA_MATRICULA', 'PROVINCIA_MATRICULA',
       'ID_SITUACION_MATRICULA', 'COLEGIO_REGISTRA', 'ACTIVO', 'COLEGIO_EMITE',
       'ID_ORIGEN_EMITE', 'ORIGEN_EMITE', 'ID_PROFESIONAL_SANCION',
       'FALLECIDO', 'prov_adjudicada', 'CODIGO_POSTAL', 'LOCALIDAD',
       'DEPARTAMENTO', 'PROVINCIA', 'Unnamed: 0.1', 'enf_max_grado',
       'prof_adjudicada', 'EDAD', 'fallecido_al_corte', 'año_corte',
       'FECHA_NACIMIENTO_y', 'FECHA_INACTIVO', 'no_jubilado_2011',
       'no_jubilado_2012', 'no_jubilado_2

In [64]:
df_sanciones.head()

,ID_PROFESIONAL_MATRICULA,mat_activa_2011,mat_activa_2012,mat_activa_2013,mat_activa_2014,mat_activa_2015,mat_activa_2016,mat_activa_2017,mat_activa_2018,mat_activa_2019,mat_activa_2020
0,21580301,True,True,True,True,True,True,True,True,True,True
1,21580336,True,True,True,True,True,True,True,True,True,True
2,21162545,True,True,True,True,True,True,True,True,True,True
3,21693334,True,True,True,True,True,True,True,True,True,True
4,21936617,True,True,True,True,True,True,True,True,True,True


# Matriz

In [65]:
df_activo_por_anio = pd.DataFrame(df_sanciones.iloc[:,-10:].to_numpy() * df_fall.iloc[:,-10:].to_numpy() * df_jubilacion.iloc[:,-10:].to_numpy()).rename({
    0:'activo_2011',
1:'activo_2012',
2:'activo_2013',
3:'activo_2014',
4:'activo_2015',
5:'activo_2016',
6:'activo_2017',
7:'activo_2018',
8:'activo_2019',
9:'activo_2020'}, axis = 1)

In [66]:
#df_activo_por_anio = df_activo_por_anio[df_activo_por_anio.iloc[:,-10:] > 0]

In [67]:
df_activo_por_anio = pd.concat([df_mat[['ID_PROFESIONAL_MATRICULA']], df_activo_por_anio], axis = 1)

In [68]:
df_activo_por_anio = pd.merge(df[['ID_PROFESIONAL','ID_PROFESIONAL_PROFESION','ID_PROFESIONAL_MATRICULA', 'FECHA_NACIMIENTO', 'FECHA_MATRICULA','prof_adjudicada', 'ID_PROFESION_REFERENCIA', 'enf_max_grado', 'prov_adjudicada', 'SEXO', 'EDAD', 'DEPARTAMENTO']], df_activo_por_anio, on = 'ID_PROFESIONAL_MATRICULA', how = 'left')

In [ ]:
df_activo_por_anio.to_csv('df_activos_anios2011-2022.csv')

In [ ]:
#df_activo_prof_por_anio.to_csv('df_activos_anios2011-2022.csv')#[df_activo_prof_por_anio['prof_adjudicada'] == 35].groupby('prov_adjudicada').sum().reset_index()#.to_csv('serie_medicos.xlsx')

In [101]:
df_activo_por_anio.loc[df_activo_por_anio['ID_PROFESION_REFERENCIA'] == 35].drop_duplicates('ID_PROFESIONAL_PROFESION').sum()

/tmp/ipykernel_25694/752398765.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_activo_por_anio.loc[df_activo_por_anio['ID_PROFESION_REFERENCIA'] == 35].drop_duplicates('ID_PROFESIONAL_PROFESION').sum()


ID_PROFESIONAL                                                  8197672164678
ID_PROFESIONAL_PROFESION                                        5346683607028
ID_PROFESIONAL_MATRICULA                                        5561128647560
prof_adjudicada                                                     9387145.0
ID_PROFESION_REFERENCIA                                               9400405
enf_max_grado                                                         23630.0
prov_adjudicada                                                     1796174.0
SEXO                        MFFMFMMMFFMMMMMMMMMFMFFFMFFMMMMMMMMMMMMFMFFFFM...
EDAD                                                                 14817297
activo_2011                                                            152905
activo_2012                                                            155522
activo_2013                                                            158042
activo_2014                                                     